In [62]:
import pandas as pd
import numpy as np
import sqlite3
import os

# 1. Check to make sure what column headers are available in my updated weight db

In [46]:
# Load a JTI weights file to check the format
jti_weights_path = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db'
weight_fn = 'JTI_Colon_Transverse.with_snp_id.db'
conn = sqlite3.connect(f'{jti_weights_path}/{weight_fn}')
# conn = sqlite3.connect('/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Whole_Blood.db')

cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print('# All tables:', cur.fetchall())
print('#', '-'*20)

cmd = "SELECT * FROM weights"
cur.execute(cmd)
print(cur.fetchone())
print(cur.fetchone())
print(cur.fetchone())
print(cur.fetchone())
print('#', '-'*20)

snp = 'chr4:76435093:T:C'
cmd = f"SELECT * FROM weights WHERE snp_id_GRch38 = ?"
res = cur.execute(cmd, (snp,)).fetchone()
print(res)
print('#', '-'*20)

print('#' * 20)
snps = ['chr4:76435093:T:C', 'chr4:163447245:T:C', 'chr4:163447245:T:C', 'chr4:186837848:C:T']
cmd = f"SELECT * FROM weights WHERE snp_id_GRch38 IN ({','.join('?'*len(snps))})"
res = cur.execute(cmd, snps).fetchall()
print(res)
print('#', '-'*20)

print([v[1] for v in cur.execute(f"PRAGMA table_info(weights)").fetchall()])


gene = 'ENSG00000164129'
cmd = f"SELECT * FROM weights WHERE gene='{gene}'"
df_weights = pd.'weights',), ('sample_info',), ('extra',), ('construction',)read_sql_query(sql=cmd, con=conn)
# df_weights

cmd = f"SELECT * FROM weights"
df_all_weights = pd.read_sql_query(sql=cmd, con=conn)
cur.close()


# All tables: [('weights',), ('sample_info',), ('extra',), ('construction',)]
# --------------------
('rs10000169', 'ENSG00000163749', 0.17842051839471315, 'T', 'C', 'chr4:77356246:T:C', 'chr4:76435093:T:C')
('rs10000239', 'ENSG00000164129', 0.0006143162235295212, 'T', 'C', 'chr4:164368397:T:C', 'chr4:163447245:T:C')
('rs10000282', 'ENSG00000250829', -0.009943939242380807, 'C', 'T', 'chr4:187759002:C:T', 'chr4:186837848:C:T')
('rs10000282', 'ENSG00000250971', -0.03620415832189286, 'C', 'T', 'chr4:187759002:C:T', 'chr4:186837848:C:T')
# --------------------
('rs10000169', 'ENSG00000163749', 0.17842051839471315, 'T', 'C', 'chr4:77356246:T:C', 'chr4:76435093:T:C')
# --------------------
####################
[('rs10000169', 'ENSG00000163749', 0.17842051839471315, 'T', 'C', 'chr4:77356246:T:C', 'chr4:76435093:T:C'), ('rs10000239', 'ENSG00000164129', 0.0006143162235295212, 'T', 'C', 'chr4:164368397:T:C', 'chr4:163447245:T:C'), ('rs10000282', 'ENSG00000250829', -0.009943939242380807, 'C', 'T'

# 2. Check the updated .db file
Make sure ref and alt alleles are matched in org db and updated SNP ids

There are some SNPs with strange ref alleles. But it is not due to my conversion.

(See code output below) The original JTI file already has those errors

For example:
* rs199942225	ENSG00000264404	0.337823	TRUE	T
* rs372054324	ENSG00000264404	0.162650	TRUE	T


In [69]:
# Load a JTI weights file to check the format
jti_weights_path = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db'

for weight_fn in os.listdir(jti_weights_path):
    if weight_fn.endswith('.db'):
        # JTI_Colon_Transverse.with_snp_id.db
        tissue = weight_fn.split('.')[0].split('JTI_')[-1]
        if 'Brain' not in weight_fn:
            continue
        print('#', tissue)
        conn = sqlite3.connect(f'{jti_weights_path}/{weight_fn}')
        cur = conn.cursor()
        
        cmd = f"SELECT * FROM weights"
        df_all_weights = pd.read_sql_query(sql=cmd, con=conn)
        
        df_all_weights = df_all_weights.dropna().copy()
        df_all_weights['ref_b37'] = df_all_weights['snp_id_GRch37'].apply(lambda x: x.split(':')[-2])
        df_all_weights['alt_b37'] = df_all_weights['snp_id_GRch37'].apply(lambda x: x.split(':')[-1])
        
        df_all_weights['ref_b38'] = df_all_weights['snp_id_GRch38'].apply(lambda x: x.split(':')[-2])
        df_all_weights['alt_b38'] = df_all_weights['snp_id_GRch38'].apply(lambda x: x.split(':')[-1])
        if len(df_all_weights[df_all_weights['ref_b37']!=df_all_weights['ref_b38']]) !=0:
            print('# - B37 ref != B38 ref')
        if len(df_all_weights[df_all_weights['alt_b37']!=df_all_weights['alt_b38']]) !=0:
            print('# - B37 alt != B38 alt')
        if len(df_all_weights[df_all_weights['ref_allele']!=df_all_weights['ref_b38']]) !=0:
            print('# - org_ref != ref_b38')
            display(df_all_weights[df_all_weights['ref_allele']!=df_all_weights['ref_b38']])
        if len(df_all_weights[df_all_weights['eff_allele']!=df_all_weights['alt_b38']]) !=0:
            print('# - org_alt != alt_b38')
            display(df_all_weights[df_all_weights['eff_allele']!=df_all_weights['alt_b38']])
        cur.close()



# Brain_Spinal_cord_cervical_c-1
# Brain_Hippocampus
# Brain_Cortex
# Brain_Nucleus_accumbens_basal_ganglia
# Brain_Caudate_basal_ganglia
# Brain_Substantia_nigra
# Brain_Anterior_cingulate_cortex_BA24
# Brain_Cerebellar_Hemisphere
# - org_ref != ref_b38


,rsid,gene,weight,ref_allele,eff_allele,snp_id_GRch37,snp_id_GRch38,ref_b37,alt_b37,ref_b38,alt_b38
37595,rs199942225,ENSG00000265203,-0.010954,C,T,chr10:48953923:TRUE:T,chr10:46785439:TRUE:T,TRUE,T,TRUE,T
37596,rs199942225,ENSG00000265763,-0.171888,C,T,chr10:48953923:TRUE:T,chr10:46785439:TRUE:T,TRUE,T,TRUE,T
55425,rs372054324,ENSG00000265203,-0.143357,C,T,chr10:48992156:TRUE:T,chr10:46747206:TRUE:T,TRUE,T,TRUE,T


# Brain_Frontal_Cortex_BA9
# Brain_Cerebellum
# - org_ref != ref_b38


,rsid,gene,weight,ref_allele,eff_allele,snp_id_GRch37,snp_id_GRch38,ref_b37,alt_b37,ref_b38,alt_b38
44627,rs199942225,ENSG00000265203,-0.064616,C,T,chr10:48953923:TRUE:T,chr10:46785439:TRUE:T,TRUE,T,TRUE,T
65620,rs372054324,ENSG00000265203,-0.214592,C,T,chr10:48992156:TRUE:T,chr10:46747206:TRUE:T,TRUE,T,TRUE,T


# Brain_Hypothalamus
# Brain_Amygdala
# Brain_Putamen_basal_ganglia


In [60]:
df_all_weights[df_all_weights['snp_id_GRch37'].isna()]

,rsid,gene,weight,ref_allele,eff_allele,snp_id_GRch37,snp_id_GRch38,ref_b37,alt_b37,ref_b38,alt_b38


In [73]:
# Load a JTI weights file to check the format
jti_weights_path = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db'
weight_fn = 'JTI_Brain_Cerebellar_Hemisphere.with_snp_id.db'
# fn = f'{jti_weights_path}/{weight_fn}'
fn = '/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Brain_Cerebellar_Hemisphere.db'
conn = sqlite3.connect(fn)
# conn = sqlite3.connect('/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Whole_Blood.db')
cur = conn.cursor()
cmd = f"SELECT * FROM weights"
df_weights = pd.read_sql_query(sql=cmd, con=conn)
cur.close()

gene = 'ENSG00000264404'
df_weights[df_weights['gene']==gene]

,rsid,gene,weight,ref_allele,eff_allele
100136,rs199942225,ENSG00000264404,0.337823,TRUE,T
100137,rs372054324,ENSG00000264404,0.162650,TRUE,T


# 2. Prepare predixcan run in AGD 250k

In [ ]:
path_genotype = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/agd_250k_ALPL_region.vcf.gz'
path_predixcan = '/data100t1/gapps/MetaXcan/software'

In [ ]:
%%bash
# conda activate py3.6
jti_weights_path=/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db
weight_fn=JTI_Whole_Blood.with_snp_id.db 

# genotype_vcf=/data100t1/share/BioVU/agd_250k/vcf-converted/agd250k_chr1.primary_pass.vcf.gz
genotype_vcf=/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/agd_250k_ALPL_region.vcf.gz

# Default --model_db_snp_key is rsid. Use the new column I created
python /data100t1/gapps/MetaXcan/software/Predict.py \
    --model_db_path ${jti_weights_path}/${weight_fn} \
    --model_db_snp_key snp_id_GRch38 \
    --vcf_genotypes ${genotype_vcf} \
    --vcf_mode genotyped \
    --prediction_output alpl.txt \
    --prediction_summary_output alpl.summary.txt \
    --only_entries ENSG00000162551 \
    --verbosity 9 \
    --throw

In [80]:
# Load a JTI weights file to check the format
jti_weights_path = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db'
weight_fn = 'JTI_Brain_Cerebellar_Hemisphere.with_snp_id.db'
# fn = f'{jti_weights_path}/{weight_fn}'
fn = '/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Brain_Cerebellar_Hemisphere.db'
conn = sqlite3.connect(fn)
# conn = sqlite3.connect('/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Whole_Blood.db')
cur = conn.cursor()
# cmd = f"SELECT DISTINCT gene FROM weights"
cmd = f"SELECT * FROM weights"
df_weights = pd.read_sql_query(sql=cmd, con=conn)
display(df_weights.head(2))
cur.close()


,rsid,gene,weight,ref_allele,eff_allele
0,rs117815334,ENSG00000000419,0.082700,C,G
1,rs2420507,ENSG00000000457,-0.037631,A,G


In [78]:
# 'weights',), ('sample_info',), ('extra',), ('construction',)

# Load a JTI weights file to check the format
jti_weights_path = '/data100t1/home/wanying/BioVU/202505_hypophosphatasia/data/harmonized_predixcan_db'
weight_fn = 'JTI_Brain_Cerebellar_Hemisphere.with_snp_id.db'
# fn = f'{jti_weights_path}/{weight_fn}'
fn = '/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Brain_Cerebellar_Hemisphere.db'
conn = sqlite3.connect(fn)
# conn = sqlite3.connect('/data100t1/share/predixcan_models/gtex_v8/JTI/JTI_Whole_Blood.db')
cur = conn.cursor()

table = 'extra'
cmd = f"SELECT * FROM {table}"
df = pd.read_sql_query(sql=cmd, con=conn)
df.head(2)

# cur.close()

,gene,genename,geneannot,pred.perf.R2,n.snps.in.model,pred.perf.pval,pred.perf.qval
0,ENSG00000000419,DPM1,protein_coding,0.046892,1,3.997963e-03,9.918923e-03
1,ENSG00000000457,SCYL3,protein_coding,0.239264,9,6.486958e-12,6.120150e-11
